# infoGAN

In [1]:
import tensorflow as tf
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


# utility func

In [2]:
# save metrics
def save_metrics(metrics, epoch=None):
    # make directory if there is not
    path = "metrics_DC"
    if not os.path.isdir(path):
        os.makedirs(path)

    # save metrics
    plt.figure(figsize=(10,8))
    plt.plot(metrics["d_loss"], label="discriminative loss", color="b")
    plt.legend()
    plt.savefig(os.path.join(path, "dloss" + str(epoch) + ".png"))
    plt.close()

    plt.figure(figsize=(10,8))
    plt.plot(metrics["g_loss"], label="generative loss", color="r")
    plt.legend()
    plt.savefig(os.path.join(path, "g_loss" + str(epoch) + ".png"))
    plt.close()

    plt.figure(figsize=(10,8))
    plt.plot(metrics["g_loss"], label="generative loss", color="r")
    plt.plot(metrics["d_loss"], label="discriminative loss", color="b")
    plt.legend()
    plt.savefig(os.path.join(path, "both_loss" + str(epoch) + ".png"))
    plt.close()

In [3]:
# plot images
def save_imgs(images, plot_dim=(10,6), size=(6,10), epoch=None):
    # make directory if there is not
    path = "generated_figures_DC"
    if not os.path.isdir(path):
        os.makedirs(path)

    num_examples = plot_dim[0]*plot_dim[1]
    num_examples = 60
    fig = plt.figure(figsize=size)

    for i in range(num_examples):
        plt.subplot(plot_dim[0], plot_dim[1], i+1)
        img = images[i, :]
        img = img.reshape((28, 28))
        plt.tight_layout()
        plt.imshow(img, cmap="gray")
        plt.axis("off")
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    plt.savefig(os.path.join(path, str(epoch) + ".png"))
    plt.close()

In [4]:
# training
import pickle
import numpy as np
import os

def unpickle(file):
    fo = open(file, 'rb')
    #print(file)
    dict = pickle.load(fo, encoding='latin1')
    fo.close()
    return dict

def one_hot_vec(label):
    vec = np.zeros(10)
    vec[label] = 1
    return vec

def load_data():
    x_all = []
    y_all = []
    for i in range (5):
        d = unpickle("cifar-10-batches-py/data_batch_" + str(i+1))
        x_ = d['data']
        y_ = d['labels']
        x_all.append(x_)
        y_all.append(y_)

    d = unpickle('cifar-10-batches-py/test_batch')
    x_all.append(d['data'])
    y_all.append(d['labels'])

    x = -0.5 + (np.concatenate(x_all) / np.float32(255))
    y = np.concatenate(y_all)
    x = np.dstack((x[:, :1024], x[:, 1024:2048], x[:, 2048:]))
    x = x.reshape((x.shape[0], 32, 32, 3))

    #pixel_mean = np.mean(x[0:50000],axis=0)
    #x -= pixel_mean
    y = np.array(list(map(one_hot_vec, y)))
    X_train = x[0:50000,:,:,:]
    Y_train = y[0:50000]
    #X_test = x[50000:,:,:,:]
    #Y_test = y[50000:]

    #return (X_train, Y_train, X_test, Y_test)
    return X_train, Y_train

# model

In [5]:
class Generator:
    def __init__(self):
        self.reuse = False
        self.initializer = tf.contrib.layers.xavier_initializer()
        self.X_dim = 28*28*1 
        self.z_dim = 100

    def __call__(self, z, training=False):
        with tf.variable_scope('g', reuse=self.reuse):
            inputs = z
            
            fc1 = tf.layers.dense(inputs, 1024, kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02)) # noise???
            fc1 = tf.nn.relu(fc1)
            
            fc2 = tf.layers.dense(fc1, 512*7*7, kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02)) # noise???
            fc2 = tf.layers.batch_normalization(fc2)
            fc2 = tf.nn.relu(fc2)
            
            fc2 = tf.reshape(fc2, [-1, 7, 7, 512])

            conv1 = tf.layers.conv2d_transpose(fc2, 256, [4,4],[2,2], padding="SAME", kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02)) # 14x14
            #conv1 = tf.layers.batch_normalization(conv1)
            conv1 = tf.nn.relu(conv1)  
            #conv1 = tf.nn.tanh(conv1)

            conv2 = tf.layers.conv2d_transpose(conv1, 1, [4,4],[2,2], padding="SAME", kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02)) # 28x28
            #conv2 = tf.layers.batch_normalization(conv2)

            #conv2 = tf.nn.tanh(conv2)
            #conv2 = tf.nn.sigmoid(conv2)
            conv2 = tf.reshape(conv2, [-1, 28*28*1])
            print(conv2.get_shape())
            outputs = conv2
        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='g')
        return outputs

In [6]:
class Discriminator:
    def __init__(self):
        self.reuse = False
        self.X_dim = 28*28*1
        self.initializer = tf.contrib.layers.xavier_initializer()

    def __call__(self, x,training=False, name=''):
        def leaky_relu(x, leak=0.2, name='outputs'):
            return tf.maximum(x, x * leak, name=name)

        with tf.name_scope('d' + name), tf.variable_scope('d', reuse=self.reuse):
            x = tf.reshape(x, [-1, 28, 28, 1])

            conv1 = tf.layers.conv2d(x, 128, [4,4], [2,2], padding="SAME", kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02)) # 14x14x64
            conv1 = leaky_relu(conv1)

            #print(conv1.get_shape())

            #conv1 = tf.concat([conv1, y], 3)

            conv2 = tf.layers.conv2d(conv1, 256, [4,4], [2,2], padding="SAME", kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02)) # 14x14
            conv2 = tf.layers.batch_normalization(conv2)
            conv2 = leaky_relu(conv2)
            conv2 = tf.contrib.layers.flatten(conv2)
            
            """
            conv3 = tf.layers.conv2d(conv2, 512, [4,4], [2,2], padding="SAME", kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02)) # 7x7
            conv3 = tf.layers.batch_normalization(conv3)
            conv3 = leaky_relu(conv3)


            conv4 = tf.layers.conv2d(conv3, 256, [4,4], [1,1], padding="SAME", kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02)) # 7x7
            conv4 = tf.layers.batch_normalization(conv4)
            conv4 = leaky_relu(conv4)
            """
            outputs = tf.layers.dense(conv2, 512, kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02))
            outputs = leaky_relu(outputs)
            #outputs = tf.nn.dropout(outputs, 0.5)
            outputs = tf.layers.dense(outputs, 1, kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02))
            #outputs = leaky_relu(outputs)
            #outputs = tf.nn.sigmoid(outputs)

        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='d')
        return outputs

In [7]:
# f(x) = ||h(x) - h(x_)|| - ||h(x)||
class Critic(object):
    def __init__(self, h):
        self.h = h
    def __call__(self, x, x_):
        return tf.norm(self.h(x) - self.h(x_), axis=1) - tf.norm(self.h(x), axis=1)

# f(x) = ||h(x) - h(x_)||
class calc_Norm(object):
    def __init__(self, h):
        self.h = h
    def __call__(self, x, x_):
        return tf.norm(self.h(x) -self.h(x_))

In [ ]:
class GAN:
    def __init__(self):
        self.batch_size = 256
        img_size = 28
        num_class = 10
        self.z_dim = 100

        self.epochs = 10000
        self.epoch_saveMetrics = 500
        self.epoch_saveSampleImg = 500
        self.epoch_saveParamter = 1000
        self.losses = {"d_loss":[], "g_loss":[]}

        #self.x, self.y = load_data()

        self.X_tr = tf.placeholder(tf.float32, shape=[None, img_size, img_size, 1])
        #self.Y_tr = tf.placeholder(tf.float32, shape=[None, num_class])
        self.Z1 = tf.placeholder(tf.float32, [None, self.z_dim])
        
        self.g = Generator()
        self.d = Discriminator()
        self.Xg = self.g(self.Z1)

    def loss_DC(self):
        Dr = self.d(self.X_tr)
        Dg = self.d(self.Xg)



        g_cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Dg, labels=tf.ones_like(Dg)))
        d_cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Dg, labels=tf.zeros_like(Dg)))
        d_cost += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Dr, labels=tf.ones_like(Dr)))
        #d_cost /= 2.

        return g_cost, d_cost

    def train(self):
        # Optimizer
        learning_rate = 1e-4
        #learning_rate = 2e-4
        beta1 = 0.5
        beta2 = 0.9

        self.L_g, self.L_d = self.loss_DC()

        d_opt = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=beta1, beta2=beta2)
        d_train_op = d_opt.minimize(self.L_d, var_list=self.d.variables)
        g_opt = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=beta1, beta2=beta2)
        g_train_op = g_opt.minimize(self.L_g, var_list=self.g.variables)

        saver = tf.train.Saver()
        #%debug

        """
        config = tf.ConfigProto(
            gpu_options=tf.GPUOptions(
                visible_device_list="0"
            )
        )
        """
        
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            epoch_pre = 0
            path = "model_DC"
            #saver.restore(sess, path+"/dcgan_model" + str(epoch_pre) + ".ckpt")
            z_test = np.random.uniform(-1, 1, size=[60, self.z_dim])
            #y_tmp = np.repeat(np.arange(10), 6)
            #y_test = np.array(list(map(one_hot_vec, y_tmp)))
            #print(y_test.shape)
            #print(y_test)

            for epoch in range(self.epochs):              
                for _ in range(1):
                    # 訓練データを抜粋
                    z1 = np.random.uniform(-1, 1, size=[self.batch_size, self.z_dim])
                    X_mb, _ = mnist.train.next_batch(self.batch_size)
                    X_mb = np.reshape(X_mb, [-1, 28, 28, 1])
                    #X_mb = (X_mb/255)

                    #rand_id = np.random.randint(0, self.x.shape[0], size=self.batch_size)
                    #X_mb = self.x[rand_id]
                    #Y_mb = self.y[rand_id]

                    # X_mb = X_mb/255
                    #X_mb = X_mb.astype(np.float32)

                    # Y_mb = Y_mb/255
                    #Y_mb = Y_mb.astype(np.float32)
                    
                    _, d_loss_value = sess.run([d_train_op, self.L_d], feed_dict={self.X_tr: X_mb, self.Z1:z1})

                # train G
                _, g_loss_value = sess.run([g_train_op, self.L_g], feed_dict={self.X_tr: X_mb, self.Z1: z1})

                # generate Sample Imgs
                #sampleImgsOfX2Y, sampleImgsOfY2X = sess.run([self.X2Y, self.Y2X], feed_dict={self.X_tr: X_mb, self.Y_tr: Y_mb})

                # 結果をappend
                self.losses["d_loss"].append(np.sum(d_loss_value))
                self.losses["g_loss"].append(np.sum(g_loss_value))
                
                if epoch % 100 == 0:
                    print("epoch:" + str(epoch+epoch_pre))

                # lossの可視化
                if epoch % self.epoch_saveMetrics == 1:
                    save_metrics(self.losses, epoch)

                # 画像の変換テスト
                if epoch % self.epoch_saveSampleImg == 0:
                    img = sess.run(self.Xg, feed_dict={self.Z1: z_test})
                    #img = img
                    #img += 0.5
                    #img *= 255
                    #print(img.shape)
                    save_imgs(img, epoch=str(epoch+epoch_pre))
                # parameterのsave
                if epoch % self.epoch_saveParamter == 1:
                    path = "model_DC"
                    if not os.path.isdir(path):
                        os.makedirs(path)

                    saver.save(sess, path+"/dcgan_model" + str(epoch+epoch_pre) + ".ckpt")
       

    def sample_images(self, row=5, col=12, inputs=None, epoch=None):
        images = self.g(inputs, training=True)
        return images

In [ ]:
gan = GAN()
gan.train()

(?, 784)
epoch:0
epoch:100
epoch:200
epoch:300
epoch:400
epoch:500
epoch:600
epoch:700
epoch:800
epoch:900
epoch:1000
epoch:1100
epoch:1200
epoch:1300
epoch:1400
epoch:1500
epoch:1600
epoch:1700
epoch:1800
epoch:1900
epoch:2000
epoch:2100
epoch:2200
epoch:2300
epoch:2400
epoch:2500
epoch:2600
epoch:2700
epoch:2800
